<a href="https://colab.research.google.com/github/srikarraju/NLP_Project/blob/main/Distant_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ADL/ADL_1/distnt_training_1.6.csv",encoding='ISO-8859-1')
print(df.columns)

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')


In [ ]:
labels = df['0'].tolist()
col_str = "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
tweets = df[col_str].tolist()
print(labels[0:10])
print(tweets[0:10])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
["is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds', 'my whole body feels itchy and like its on fire ', "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. ", '@Kwesidei not the whole crew ', 'Need a hug ', "@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?", "@Tatiana_K nope they didn't have it ", '@twittera que me muera ? ', "spring break in plain city... it's snowing "]


In [ ]:
from gensim.models import Word2Vec,KeyedVectors
import gensim.downloader
#model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Colab Notebooks/NLP/GoogleNews_Vectors_300.bin', binary=True)
model = gensim.downloader.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [ ]:
def get_train_sentence_vectors(sentences,model):
  words_to_integer_dict = dict()
  integer_to_word_dict = dict()
  print("No. of sentences = ",len(sentences))
  sentence_vectors = []
  max_sentence_length = 0
  word_count = 1
  for sent in sentences:
    vec = []
    sent_length = 0
    for word in sent:
      sent_length += 1
      if words_to_integer_dict.get(word) == None:
        if word in model:
          words_to_integer_dict[word] = word_count
          vec.append(word_count)
          word_count += 1
        else:
          vec.append(0)
      else:
        vec.append(words_to_integer_dict[word])
    max_sentence_length = max(sent_length,max_sentence_length)
    sentence_vectors.append(vec)
  return sentence_vectors,word_count,max_sentence_length,words_to_integer_dict

In [ ]:
import numpy as np
def get_initial_embeddings(model_ft,word_count,words_to_integer_dict):
  count_in_model = 0
  embedding_matrix = [[0]*50]*(word_count+2)  #one for not in model one for not in train data
  for word in words_to_integer_dict:
    if word in model_ft:
      count_in_model += 1
      embedding_matrix[words_to_integer_dict[word]] = model[word]
  embedding_matrix = np.asarray(embedding_matrix,dtype=float)
  print("No. of embeddings in model = ",count_in_model)
  return embedding_matrix

In [ ]:
def pad_input_vectors(sentence_vectors,max_sentence_length):
  for i in range(len(sentence_vectors)):
    sentence_vectors[i] = sentence_vectors[i][0:max_sentence_length]
    for j in range(max_sentence_length-len(sentence_vectors[i])):
      sentence_vectors[i].append(0)
  #print(sentence_vectors[0:2])
  data_x = np.asarray(sentence_vectors)
  return data_x

In [ ]:
# CNN model
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Reshape, Input, Embedding, Dense, Softmax
from keras.models import Sequential,Model
import tensorflow as tf

def get_trained_CNN_model(train_x,train_y,embedding_matrix,word_count,max_sentence_length):
  initializer = tf.keras.initializers.HeNormal()
  input = Input(shape=(max_sentence_length,))
  e = Embedding(word_count+2, 50, weights = [embedding_matrix], input_length=max_sentence_length)(input)
  e.trainable = False
  reshaped_embedding = Reshape((1,max_sentence_length,50,1))(e)
  filter_2 = Conv2D(50,(2,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding)
  filter_3 = Conv2D(50,(3,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding) 
  filter_4 = Conv2D(50,(4,50),activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(reshaped_embedding)
  reshaped_filter_2 = Reshape((max_sentence_length-1,50,1))(filter_2)
  reshaped_filter_3 = Reshape((max_sentence_length-2,50,1))(filter_3)
  reshaped_filter_4 = Reshape((max_sentence_length-3,50,1))(filter_4)
  max_pooling_2 = MaxPooling2D(pool_size=(max_sentence_length-1,1))(reshaped_filter_2)
  max_pooling_3 = MaxPooling2D(pool_size=(max_sentence_length-2,1))(reshaped_filter_3)
  max_pooling_4 = MaxPooling2D(pool_size=(max_sentence_length-3,1))(reshaped_filter_4)
  concatenated_output = Concatenate()([max_pooling_2,max_pooling_3,max_pooling_4])
  concatenated_output_reshaped = Reshape((1,150))(concatenated_output)
  dense1_output = Dense(50,activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(concatenated_output_reshaped)
  dense2_output = Dense(3,activation='relu',kernel_initializer=initializer,bias_initializer=initializer,
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=tf.keras.regularizers.l2(1e-4))(dense1_output)
  softmax_probs = Softmax()(dense2_output)

  model3 = Model(input,softmax_probs)
  print(model3.summary())
  print("compiling model")
  model3.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  print("fitting model")
  model3.fit(train_x,train_y,epochs=2)
  e.trainable = True
  model3.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
  model3.fit(train_x,train_y,epochs=2)
  return model3

In [ ]:
seen_1,seen_2 = 0,0
for i in range(len(labels)):
  if labels[i] == 4 and seen_1==0:
    seen_1 = 1
    print(i)
    labels[i]=1

799999


In [ ]:
tweets1 = tweets[0:100000]
tweets1 += tweets[799999:899999]

In [ ]:
import numpy as np
labels1 = labels[0:100000]
labels1 += [2]*100000
labels1 = np.asarray(labels1)

In [ ]:
for i in range(len(tweets1)):
  tweets1[i] = tweets1[i].split(" ")

In [ ]:
sentence_vectors,word_count,max_sentence_length,words_to_integer_dict = get_train_sentence_vectors(tweets1,model)

data_x = pad_input_vectors(sentence_vectors,max_sentence_length)

embedding_matrix = get_initial_embeddings(model,word_count,words_to_integer_dict)

No. of sentences =  200000
No. of embeddings in model =  38969


In [ ]:
print(tweets1[0])

['is', 'upset', 'that', 'he', "can't", 'update', 'his', 'Facebook', 'by', 'texting', 'it...', 'and', 'might', 'cry', 'as', 'a', 'result', '', 'School', 'today', 'also.', 'Blah!']


In [ ]:
CNN_model = get_trained_CNN_model(data_x,labels1,embedding_matrix,word_count,max_sentence_length)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 110)]        0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 110, 50)      1948600     input_4[0][0]                    
__________________________________________________________________________________________________
reshape_15 (Reshape)            (None, 1, 110, 50, 1 0           embedding_3[0][0]                
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 1, 109, 1, 50 5050        reshape_15[0][0]                 
____________________________________________________________________________________________

In [ ]:
embeddings = CNN_model.layers[1].get_weights()[0]

In [ ]:
import pandas as pd

words = []
word_embeddings = []
for each in words_to_integer_dict:
  words.append(each)
  word_embeddings.append(embeddings[words_to_integer_dict[each]])

output_df = pd.DataFrame({'words':words,'embeds':word_embeddings},index=None)
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Project/distant_training_embeddings.csv',index=False) 


In [ ]:
print(embeddings[words_to_integer_dict['good']])

[ 0.9088729  -0.38718292 -0.18096112 -0.02151196  0.556913    0.9932972
  1.5273864   0.31223163 -0.9377556   0.19941017 -0.96523255  0.48952287
 -4.695946   -0.28799155 -0.08227208  0.06594674  0.62876326 -0.11122999
 -1.1907783   0.4457765  -0.8488463  -0.17991054 -0.29931265  0.77638113
  0.33496284  0.74343866  0.61681217 -0.02405708  1.0758072  -0.52666056
 -0.30289677  0.39240065 -0.0980799   0.47156036  0.60399926  0.7055437
  0.19352466  0.04299197  0.01745365  1.5401787  -1.2847718  -0.31853494
  0.800926    0.493639    0.3038349   0.67784417 -0.22689356 -0.28241208
 -0.0377995   0.21563646]


In [ ]:
print(model.wv['good'])

[ 0.6608   -0.10159   0.026775 -0.088053  0.15578   0.87288   1.29
  0.28934  -0.59205   0.26779  -0.76604   0.27955  -5.1483   -0.056899
 -0.050798 -0.083225  0.48048  -0.35409  -1.0566    0.065436 -0.46674
  0.13847  -0.22022   0.61591   0.18462   0.77965   0.29022  -0.24679
  0.95335  -0.35699  -0.24246   0.35939  -0.16369   0.30926   0.32784
  0.66924  -0.028869  0.13981   0.12371   0.96181  -1.4018   -0.19285
  0.79053   0.36647   0.32751   0.29666  -0.039173 -0.14523  -0.19663
  0.026827]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
print(embeddings[words_to_integer_dict['bad']])

[ 0.6667318   0.17410867 -0.76818365 -0.14902386 -0.5615814   0.13316992
  1.2899168   0.51425874  0.20434889  0.9541428   0.1850975  -0.46439135
 -4.665776    0.11483898  0.3595791  -0.09103412  0.23399332 -0.37052622
 -0.82646435 -0.69965285 -0.27986917  0.7830526   0.68211406  0.7676632
  0.10351181  0.7002878  -0.33383143 -0.18528323 -0.84498984 -0.31981003
 -0.7581209  -0.34977284 -0.06020527 -0.36154577 -0.6455438   0.66562253
 -1.1050513  -0.2788534   0.07307023  0.23305412 -0.95882726  0.54246455
  0.34790987  0.03235342 -0.5768586  -0.7598304   0.96125615  0.9636172
 -0.17310408  0.6728128 ]


In [ ]:
import pandas as pd
import numpy as np
def get_embeds_matrix(embeds_list):
  embeds_matrix = []
  for i in range(len(embeds_list)):
    embeds_list[i] = str(embeds_list[i][1:len(embeds_list[i])-1])
    array = embeds_list[i].split()
    for j in range(0,len(array)):
      array[j] = float(array[j])
    embeds_matrix.append(array)
  
  return np.asarray(embeds_matrix)

embeds_file = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/Project/distant_training_embeddings.csv")
words_list = embeds_file['words'].tolist()
embeds_list = embeds_file['embeds'].tolist()
embeds_matrix = get_embeds_matrix(embeds_list)

In [ ]:
import math
def cosine_similarity(vec_1,vec_2):
  norm1 = 0
  for i in range(len(vec_1)):
    norm1 += vec_1[i]**2
  norm2 = 0
  for i in range(len(vec_2)):
    norm2 += vec_2[i]**2
  norm1 = math.sqrt(norm1)
  norm2 = math.sqrt(norm2)
  cosine = 0
  for i in range(len(vec_1)):
    cosine += vec_1[i]*vec_2[i]
  return (cosine/(norm1*norm2))

In [ ]:
print(cosine_similarity(embeds_matrix[words_list.index('good')],embeds_matrix[words_list.index('amazing')]))

0.7863735700087224


In [ ]:
print(cosine_similarity(model.wv['good'],model.wv['amazing']))

0.8494970125985905


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2).fit_transform(embeds_matrix)

In [ ]:
print(X_embedded[words_list.index('good')])
print(X_embedded[words_list.index('bad')])

[52.84215  33.566704]
[59.97386  18.976067]
